In [308]:
infilename = '../../H1B_FY_2016.csv'

In [309]:
states = ['HI','AK','WA','OR','CA','ID','NV','AZ','UT','WY','MT'\
          ,'CO','NM','TX','OK','KS','NE','SD','ND','MN','IA','MO','AR','LA','MS'\
          ,'TN','KY','IL','WI','MI','IN','OH','AL','FL','GA','SC','NC','VA','WV'\
          ,'DC','MD','NJ','CT','RI','MA','PA','NY','VT','NH','ME']
visa_classes = ['H-1B','H-1B1','E-3']

In [310]:
len(states)

50

In [311]:
import pandas as pd

In [312]:
indf = pd.read_csv(infilename,sep=';')

/Users/rmartinshort/anaconda/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [313]:
indf['VISA_CLASS'].unique()

array(['H-1B', 'H-1B1 Chile', 'H-1B1 Singapore', 'E-3 Australian',
       'Select Visa Classification'], dtype=object)

In [318]:
colnames = ['STATUS','VISA_CLASS','SOC_CODE','SOC_NAME','WORKLOC1_STATE','WORKSITE_STATE']

In [319]:
with open(infilename) as f:
    first_line = f.readline()
delim = first_line[0]

In [320]:
first_line = first_line.split(delim)

In [321]:
first_line

['',
 'CASE_NUMBER',
 'CASE_STATUS',
 'CASE_SUBMITTED',
 'DECISION_DATE',
 'VISA_CLASS',
 'EMPLOYMENT_START_DATE',
 'EMPLOYMENT_END_DATE',
 'EMPLOYER_NAME',
 'EMPLOYER_ADDRESS',
 'EMPLOYER_CITY',
 'EMPLOYER_STATE',
 'EMPLOYER_POSTAL_CODE',
 'EMPLOYER_COUNTRY',
 'EMPLOYER_PROVINCE',
 'EMPLOYER_PHONE',
 'EMPLOYER_PHONE_EXT',
 'AGENT_ATTORNEY_NAME',
 'AGENT_ATTORNEY_CITY',
 'AGENT_ATTORNEY_STATE',
 'JOB_TITLE',
 'SOC_CODE',
 'SOC_NAME',
 'NAIC_CODE',
 'TOTAL_WORKERS',
 'FULL_TIME_POSITION',
 'PREVAILING_WAGE',
 'PW_UNIT_OF_PAY',
 'PW_WAGE_SOURCE',
 'PW_SOURCE_YEAR',
 'PW_SOURCE_OTHER',
 'WAGE_RATE_OF_PAY_FROM',
 'WAGE_RATE_OF_PAY_TO',
 'WAGE_UNIT_OF_PAY',
 'H-1B_DEPENDENT',
 'WILLFUL_VIOLATOR',
 'WORKSITE_CITY',
 'WORKSITE_COUNTY',
 'WORKSITE_STATE',
 'WORKSITE_POSTAL_CODE',
 'ORIGINAL_CERT_DATE\n']

In [322]:
#Figure out which columns to use

used_colnames = []
used_colname_indices = []

for name in colnames:
    for header in first_line:
        if name in header:
            used_colnames.append(header)
            used_colname_indices.append(first_line.index(header))
            
print(used_colnames,used_colname_indices)

['CASE_STATUS', 'VISA_CLASS', 'SOC_CODE', 'SOC_NAME', 'WORKSITE_STATE'] [2, 5, 21, 22, 38]


In [323]:
infile = open(infilename)
all_lines = infile.readlines()
infile.close()

#Generate dictionary to fill with names
alldata = {}
for name in used_colnames:
    alldata[name] = ['NAN']*(len(all_lines)-1)

In [324]:
list(alldata.keys())

['CASE_STATUS', 'VISA_CLASS', 'SOC_CODE', 'SOC_NAME', 'WORKSITE_STATE']

In [325]:
%%time

i = 0
total_certified = 0

db = open('for_debugging.txt','w')
for line in all_lines[1:]:
    vals = line.split(delim)
    status = vals[used_colname_indices[0]]
    visa_class = vals[used_colname_indices[1]].split(' ')[0]
    soc_code = vals[used_colname_indices[2]]
    soc_name = vals[used_colname_indices[3]]
    workstate = vals[used_colname_indices[4]].strip()
    if (status == "CERTIFIED") and (len(soc_code)==7) and (workstate in states):
            #print(status,visa_class,soc_code,soc_name,workstate)
            alldata[used_colnames[0]][i] = status
            alldata[used_colnames[1]][i] = visa_class
            alldata[used_colnames[2]][i] = soc_code
            alldata[used_colnames[3]][i] = soc_name
            alldata[used_colnames[4]][i] = workstate
            total_certified += 1 
    db.write('%s;%s;%s;%s;%s\n' %(alldata[used_colnames[0]][i],alldata[used_colnames[1]][i],\
        alldata[used_colnames[2]][i],alldata[used_colnames[3]][i],alldata[used_colnames[4]][i]))
    i += 1

db.close()
#Remove NANS
#for colname in used_colnames:
#    alldata[colname][:] = [value for value in alldata[colname] if value != 'NAN']

CPU times: user 3.6 s, sys: 60.5 ms, total: 3.66 s
Wall time: 3.69 s


In [326]:
print(total_certified)

558533


In [327]:
test = pd.read_csv('for_debugging.txt',sep=';',names=['status','type','code','job','state'])

In [328]:
import numpy as np
test.replace('NAN', np.nan,inplace=True)

In [329]:
test.dropna(inplace=True)

In [330]:
test['state'].unique()

array(['CA', 'VA', 'TX', 'FL', 'DC', 'IL', 'NC', 'RI', 'OH', 'MI', 'GA',
       'NY', 'NJ', 'WA', 'MA', 'IN', 'MD', 'PA', 'MO', 'NV', 'MN', 'AZ',
       'CO', 'TN', 'MS', 'UT', 'KS', 'WI', 'CT', 'NE', 'OK', 'AR', 'KY',
       'SC', 'HI', 'AK', 'IA', 'WY', 'LA', 'NM', 'WV', 'SD', 'OR', 'ND',
       'ME', 'AL', 'VT', 'ID', 'NH', 'MT'], dtype=object)

In [331]:
test['status'].unique()

array(['CERTIFIED'], dtype=object)

In [332]:
test['type'].unique()

array(['H-1B', 'H-1B1', 'E-3'], dtype=object)

In [333]:
len(test['code'].unique())

516

In [334]:
len(test['job'].unique())

784

In [335]:
#Find number of unique LCA SOC codes
result = {}
for entry in alldata[used_colnames[2]]:
    if (entry != 'NAN'):
        if entry not in result:
            result[entry] = 0
        result[entry] += 1

In [336]:
result['15-1121']

103588

In [337]:
#Top 10 occupations
for w in list(sorted(result, key=result.get, reverse=True))[:10]:
  ind_w = alldata[used_colnames[2]].index(w)
  ind_name = alldata[used_colnames[3]][ind_w]
  print(w, ind_name, result[w], result[w]/total_certified)

15-1132 SOFTWARE DEVELOPERS, APPLICATIONS 105603 0.1890720870566287
15-1121 COMPUTER SYSTEMS ANALYSTS 103588 0.18546442197685722
15-1131 COMPUTER PROGRAMMERS 73520 0.13163053928774127
15-1199 COMPUTER OCCUPATIONS, ALL OTHER 52634 0.09423615077354426
15-1133 SOFTWARE DEVELOPERS, SYSTEMS SOFTWARE 19232 0.03443305946112405
13-1111 MANAGEMENT ANALYSTS 13558 0.024274304293569048
15-1142 NETWORK AND COMPUTER SYSTEMS ADMINISTRATORS 10114 0.018108151174594876
13-2011 ACCOUNTANTS AND AUDITORS 9596 0.017180721640440225
17-2141 MECHANICAL ENGINEERS 8255 0.014779789197773454
13-2051 FINANCIAL ANALYSTS 7914 0.014169261261196742


In [254]:
#Find number of unique states
result = {}
for entry in alldata[used_colnames[4]]:
    if (entry != 'NAN'):
        if entry not in result:
            result[entry] = 0
        result[entry] += 1

In [255]:
#Top 10 states
for w in list(sorted(result, key=result.get, reverse=True))[:10]:
  print(w, result[w], result[w]/total_certified)

CA 96878 0.18261812081169473
TX 53825 0.10146184224167994
NY 43886 0.08272651014618422
NJ 42290 0.07971799922713692
IL 28933 0.05453962808320531
GA 20387 0.03843014543021141
MA 19996 0.0376930979556829
PA 19862 0.037440503680524796
WA 18493 0.03485989500372294
FL 17710 0.03338391502276176


In [280]:
alldata.keys()

dict_keys(['CASE_STATUS', 'VISA_CLASS', 'SOC_CODE', 'SOC_NAME', 'WORKSITE_STATE'])

In [338]:
a = '/usr/bin/env/python/hello.py'

In [340]:
'/'.join(a.split('/')[:-1])

'/usr/bin/env/python'

In [348]:
names = ['EPSILON HELLO','ALPHA GO','BETA','GAMMA']
vals = [1,1,1,3]

In [349]:
for element in sorted(zip(names,vals)):
    print(element)

('ALPHA GO', 1)
('BETA', 1)
('EPSILON HELLO', 1)
('GAMMA', 3)
